# From *_picked.hdf5 to kinetic properties for immobile particles
This notebook will run ... 
* spt.immobile_props.main()

This notebook extracts the kinetics properties in the immobilized case (TPP,...). We will therefore:
1. Define the full paths to the *_picked.hdf5 files
2. Set the execution parameters
3. Connect or start a local dask parallel computing cluster
4. Run improps.main() for all defined datasets

As a result a file with extension *_tprops.hdf5 will be created in the same folder as the *_picked.hdf5 file

In [5]:
import os
import sys
import traceback
import importlib
from dask.distributed import Client
import multiprocessing as mp

import picasso_addon.io as addon_io
import spt.immobile_props as improps

### 1. Define the full paths to the *_picked.hdf5 files

In [2]:
dir_names=[]
dir_names.extend([r'C:\Data\p06.SP-tracking\20-01-17_fix_slb_L_T21\id140_L_exp200_p114uW_T21_1'])

file_names=[]
file_names.extend(['id140_L_exp200_p114uW_T21_1_MMStack_Pos2.ome_locs_render_picked.hdf5'])

### 2. Set the execution parameters

In [3]:
### Valid for all evaluations
params_all={'filter':'fix'}

## Exceptions
params_special={}

To check for all possible parameters run `help(improps.main)`. If not stated otherwise standard values are used (indicated in brackets).

In [7]:
help(improps.main)

Help on function main in module spt.immobile_props:

main(locs, info, path, **params)
    Cluster detection (pick) in localization list by thresholding in number of localizations per cluster.
    Cluster centers are determined by creating images of localization list with set oversampling.
    
    
    args:
        locs(pd.Dataframe):        Picked localizations as created by picasso render
        info(list(dict)):          Info to picked localizations
        path(str):                 Path to _picked.hdf5 file.
        
    **kwargs: If not explicitly specified set to default, also when specified as None
        ignore(int=1):             Ignore value for bright frame
        parallel(bool=True):       Apply parallel computing using dask? 
                                   Dask cluster should be set up and running for best performance!
        filter(string='paint'):    Which filter to use, either None, 'paint' or 'fix'
        save_picked(bool=False):   If true _picked file conta

### 3. Connect or start a local dask parallel computing cluster
This is only necessary if you want to use parallel computing (standard). If not set `params_all={'parallel':False}`

In [9]:
try:
    client = Client('localhost:8787')
    print('Connecting to existing cluster...')
except OSError:
    improps.cluster_setup_howto()

Connecting to existing cluster...


If we execute the prompt (see below) a local cluster is started, and we only have to execute the cell above  to reconnect to it the next time. If you try to create a new cluster under the same address this will throw an error!

In [6]:
Client(n_workers=max(1,int(0.8 * mp.cpu_count())),
       processes=True,
       threads_per_worker=1,
       scheduler_port=8787,
       dashboard_address=":1234")

Client Scheduler: tcp://127.0.0.1:8787 Dashboard: http://127.0.0.1:1234/status,Cluster Workers: 3 Cores: 3 Memory: 16.80 GB


### Run improps.main() for all datasets

In [8]:
failed_path=[]
for i in range(0,len(file_names)):
    ### Create path
    path=os.path.join(dir_names[i],file_names[i])
    ### Set paramters for each run
    params=params_all.copy()
    for key, value in params_special.items():
        params[key]=value[i]
    
    ### Run main function
    try:
        locs,info=addon_io.load_locs(path)
        out=improps.main(locs,info,path,**params)
    except Exception:
        traceback.print_exc()
        failed_path.extend([path])

print()    
print('Failed attempts: %i'%(len(failed_path)))

Calculating kinetic information ...
... in parallel
... Computation time 151.8 s
Filtering ..(fix)
Saving _tprops ...

Failed attempts: 0
